In [2]:
import numpy as np
import pandas as pd

In [5]:
trajectories_df = pd.read_csv('../data/raw/trajectories_df-20240503.csv',
                              index_col=0,
                              dtype = {'FlightID': str, 'Bird': str, 'Age': str,
                                       'Category':str,'Iteration':int,'Time':float,
                                       'Frame': int,'X':float,'Y':float,'Z':float})      